In [1]:
from datamender import Mender

data_src = "https://media.geeksforgeeks.org/wp-content/uploads/nba.csv"

mender = Mender()

In [2]:
mender.read_csv(data_src)

In [6]:
mender.set_focus(rows=range(0,100), columns=['Name', 'Team'])

In [7]:
mender.focused.df

,Name,Team
0,Avery Bradley,Boston Celtics
1,Jae Crowder,Boston Celtics
2,John Holland,Boston Celtics
3,R.J. Hunter,Boston Celtics
4,Jonas Jerebko,Boston Celtics
...,...,...
95,Jeff Green,Los Angeles Clippers
96,Blake Griffin,Los Angeles Clippers
97,Wesley Johnson,Los Angeles Clippers
98,DeAndre Jordan,Los Angeles Clippers
